<a href="https://colab.research.google.com/github/jisusP/Data_science/blob/main/Proyecto_informes_juan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import os
from natsort import natsorted
import glob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 48.9 MB/s eta 0:00:00


In [70]:
import fitz  # PyMuPDF
from pathlib import Path

# Función para extraer texto de un PDF
def extract_text_from_pdf(pdf_path):
    # Abre el documento PDF
    document = fitz.open(pdf_path)
    text = ""
    # Itera sobre cada página
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# Ruta del directorio
directorio = "/content/"
patron_pdf = "*.pdf"

dicc={}
# Lista de archivos .pdf en el directorio
archivos_pdf = natsorted(glob.glob(directorio + patron_pdf))
for i, j in enumerate(archivos_pdf):
  dicc[i] = extract_text_from_pdf(j)

# Lista de archivos .txt en el directorio de manera individual
archivos_txt = natsorted(Path(directorio).glob(patron_pdf))
# Obtener solo los nombres de los archivos (sin extensión)
nombres_archivos = [archivo.stem for archivo in archivos_txt]

print(dicc)

{0: '                               \n \n \n \n \n \n \n \n \n \nDETERMINACIÓN DE LA ESTEQUIMETRIA DE UNA REACCIÓN \nQUÍMICA   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nAndrea Rojas, Eileen Salas & Michelle Varela  \nMayo 2024. \nUniversidad del Atlántico. \nFacultad de Ingeniería.  \nQuímica General I. \n2 \n \n \n \n \n \n \nResumen \n \nEste informe de laboratorio se basa en un experimento que explora la \nestequiometría de las reacciones químicas. De este modo, determina las cantidades de los \nreactantes y productos presentes en la misma. Durante el experimento, observamos \nmanifestaciones durante la mezcla de volúmenes, como la formación de precipitados. El \nenfoque central de este experimento fue mostrar algunos principios del análisis \ngravimétrico. Además, su utilidad radica en presentar una técnica general para recoger \ncuantitativamente un precipitado y pesarlo. \nDurante la ejecución del experimento, se observó el razonamiento estequiométrico \nen

In [71]:
def preprocess(text):
    #Texto en minúscula.
    text = text.lower()
    #Toma cualquier carácter alfanumérico y lo convierte en un espacio en blanco. Eliminar signos de puntuación.
    text = re.sub(r'\W', ' ', text)
    #Elimina secuencias de espacio en blanco para dejar una sola separación.
    text = re.sub(r'\s+', ' ', text)
    #tokeniza el texto
    tokens = nltk.word_tokenize(text)
    #Se crea el modelo de lematización.
    lemmatizer = WordNetLemmatizer()
    #Teniendo en cuenta los tokens pasados, se lematizan cada token en la lista.
    #El condicional es para no poner la palabra guía de finalización
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    #Devuelve el texto procesado en una cadena de texto unidos por espacios.
    return ' '.join(tokens)

#preprocess("Había un pato est@acionado 123")

In [72]:
# Ejemplo de cómo estructurar 'all_texts'
reference_report = [dicc[0]]
student_reports = [[pdf_est] for i, pdf_est in dicc.items() if pdf_est not in reference_report]

print(reference_report)
print(student_reports)

# Combina todos los informes en una lista
all_texts = [reference_report] + student_reports
print(all_texts)
#print(all_texts[0])
preprocessed_texts = [preprocess(text[0]) for text in all_texts]

vectorizer = TfidfVectorizer() #se llama el método TF-IDF
X_train = vectorizer.fit_transform(preprocessed_texts) #Se aplica el método TF-IDF en la lista que tiene todos los textos procesados.

['                               \n \n \n \n \n \n \n \n \n \nDETERMINACIÓN DE LA ESTEQUIMETRIA DE UNA REACCIÓN \nQUÍMICA   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nAndrea Rojas, Eileen Salas & Michelle Varela  \nMayo 2024. \nUniversidad del Atlántico. \nFacultad de Ingeniería.  \nQuímica General I. \n2 \n \n \n \n \n \n \nResumen \n \nEste informe de laboratorio se basa en un experimento que explora la \nestequiometría de las reacciones químicas. De este modo, determina las cantidades de los \nreactantes y productos presentes en la misma. Durante el experimento, observamos \nmanifestaciones durante la mezcla de volúmenes, como la formación de precipitados. El \nenfoque central de este experimento fue mostrar algunos principios del análisis \ngravimétrico. Además, su utilidad radica en presentar una técnica general para recoger \ncuantitativamente un precipitado y pesarlo. \nDurante la ejecución del experimento, se observó el razonamiento estequiométrico \nen ca

In [73]:
def calculate_similarity(student_report, reference_report):
    student_vector = vectorizer.transform([preprocess(student_report[0])]) #Se obtiene el vector TF-IDF del informe del estudiante
    reference_vector = vectorizer.transform([preprocess(reference_report[0])]) #Se obtiene el vector TF-IDF del informe del profesor
    similarity = cosine_similarity(student_vector, reference_vector) #Se le aplica la similitud del coseno a los 2 vectores
    return similarity[0][0]

def similarity_to_grade(similarity):
    return 1 + 4 * similarity

def grade_student_report(student_report):
    similarity = calculate_similarity(student_report, reference_report)
    grade = similarity_to_grade(similarity)
    return grade

In [76]:
# Ejemplo de uso
for i, report in enumerate(student_reports):
    grade = grade_student_report(report)
    print(f"Calificación del informe del estudiante {i+1}: {grade:.2f}, {nombres_archivos[i+1]}")

Calificación del informe del estudiante 1: 4.53, EXPERIENCIA N° 7
Calificación del informe del estudiante 2: 4.69, Tecnica N°7 Lab
Calificación del informe del estudiante 3: 4.68, informe laboratorio-7
